In [ ]:
import os

import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

from attacks import GM
from classifiers import MNIST_CNN
from trainers import TrainerForVAEGAN
from utils import params_maker

os.environ["CUDA_VISIBLE_DEVICES"] = "2"

fmnist = input_data.read_data_sets("FMNIST_data/")

data_train = (np.reshape(fmnist.train.images, [-1, 28, 28, 1]) * 2.0 - 1, fmnist.train.labels)
data_test = (np.reshape(fmnist.test.images, [-1, 28, 28, 1]) * 2.0 - 1, fmnist.test.labels)


In [ ]:
use_batchnorm = False
droprate = None
snbeta = None
l2rate = None

attack = GM

attack_paramss = params_maker(['eps', 'step_size', 'n_steps', 'norm', 'loss_type'],
                              [[0.3], [0.02], [40], ['inf'], ['xent', 'cw']])

for attack_params in attack_paramss:

    logdir = 'tf_logs/exp2/'
    if use_batchnorm: logdir += 'batchnorm/'
    if droprate: logdir += 'dropout/{}/'.format(droprate)
    if snbeta: logdir += 'sn/{}/'.format(snbeta)
    if l2rate: logdir += 'l2/{}/'.format(l2rate)

    if attack: logdir += 'adv/{}/{}/'.format(attack.__name__, '_'.join([str(v) for k, v in attack_params]))

    if logdir.split('/')[-2] == 'exp2': logdir += 'standard/'

    print(logdir)

    tf.reset_default_graph()

    DNN = MNIST_CNN(logdir, tf.nn.relu, use_batchnorm, droprate, snbeta, l2rate, attack, dict(attack_params))

    sess = tf.InteractiveSession()
    sess.run(tf.global_variables_initializer())

    trainer = TrainerForVAEGAN(sess, DNN, data_train)
    trainer.train(n_epochs=5, p_epochs=1)

    sess.close()

    # Test Accuracy
    tf.reset_default_graph()

    sess = tf.InteractiveSession()

    DNN = MNIST_CNN(logdir, tf.nn.relu)
    DNN.load(sess)

    print('Test Accuracy : {:.5f}'.format(DNN.evaluate(sess, data_test)))

    sess.close()
